In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'cord length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Multiple Buckets"""

# regex_pattern = r'(?i)([^d]\smeasures\s?\d+.?\"\s?L)|([^d]\smeasures\s\d+\.\d*.?\"\s?L)|()'
# regex_pattern = r'(?i)(\s?\d+.?\"\s?L)|(\s\d+\.\d*.?\"\s?L)|()'
# regex_pattern = r'(?i)(cord\s?measures\s?\d*\'\s?L?)|(cord\s?measures\s?\d*.?inch)|(cord\s?measures\s?\d*.?foot)|(cord\s?measures\s?\d*.?.?\d*)|(cord\s?length\s?\d*\'\s?L?)|(cord\s?length\s?\d*.?inch)|(cord\s?length\s?\d*.?foot)|(cord\s?length\s?\d*.?.?\d*)|()' 
# regex_pattern = r'(\d*.?Inch .?.?.?.?.?.?cord)|(\d+.? cord weighs)|(cord\s?measure.?\s?\d*\\?\"?\s?L?)|(cord\s?measure.?\s?\d*\\?\'?\s?L?)|(cord\s?measure.?\s?\d*.?inch)|(cord\s?measure.?\s?\d*.?foot)|(cord\s?measure.?\s?\d*.?.?\s?L)|(cord\s?mesure\s?\d*\'\s?L?)|(cord\s?mesure\s?\d*.?inch)|(cord\s?mesure\s?\d*.?foot)|(cord\s?mesure\s?\d*.?.?\d*)|(cord\s?measusre\s?\d*\'\s?L?)|(cord\s?measusre\s?\d*.?inch)|(cord\s?measusre\s?\d*.?foot)|(cord\s?measusre\s?\d*.?.?\d*)|(cord\s?length\s?\d*\'\s?L?)|(cord\s?length\s?\d*.?inch)|(cord\s?length\s?\d*.?foot)|(cord\s?length\s?\d*.?.?\d*)|()'     
regex_pattern_inch = r'(?i)((?<!(H Lampshade height: 7 \"))\scord\s?l?e?n?g?t?h?\s?m?e?a?s?u?r?e?.?\s?i?s?\s?\d+\s?\\?\")|((?<!(measures))(?<!(construction))\s\d*\s?\\?\"\s?L?\s?cord\s?l?e?n?g?t?h?(?>!(measures)))|()'    
regex_pattern_foot = r'(?i)((?<!(Lampshade height: 7 \"))(?<!(String measures))(?<!(any standard outlet 5.?.?.?))(?<!(Lampshade height: 10\)))(?<!(lamp is 23\"))(?>!(measures))\s\d+\'\s?cord\s?length)|(\d*\'\s?L\s?cord)|(cord\sl?e?n?g?t?h?\s?m?e?a?s?u?r?e?.?\s?\d*\s?\\?\')|()' 
regex_pattern_na = r'(?i)(cordless)|()'  



value='%n/a%'
customer_name='%bedbathandbeyond%'
strategy_version_input=668
attribute_id_input=4506


params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)

In [6]:
len(df)

4832

In [7]:
df = df[df['bucket_id'].isin(buckets)]
df

product_id external_id  \
0        4672200    76665915   
1      817479867    63228465   
2      817192887    63228441   
3      817188038    62520409   
4      817476446    62520393   
...          ...         ...   
4827  1202939208    45618345   
4828      170436    42405344   
4829      167019    18722070   
4830   988575025    40050881   
4831   817459018    42607793   

                                                                                         product_name  \
0                                                           Adesso Rodeo Floor Lamp in Antique Bronze   
1                                              Simple Designs Porcelain Puppy Dog Table Lamp in White   
2                                                  Simple Designs Porcelain Bunny Table Lamp in White   
3                                                              Décor Therapy Knox Table Lamp in Brown   
4                                                          Décor Therapy Campbell Table Lamp in Brown   
...                                                                                               ...   
4827                                                                       Circus Table Lamp in White   
4828                              Liliana 1-Light Wall Sconce in Seafoam/Aged Silver with White Shade   
4829  ELK Lighting Refraction Collection 1-Light Sconce In Polished Chrome with Creme and White Glass   
4830                                                             Studio 3B™ 18-LED Clip Lamp in Black   
4831                                                           cloud b® Sleep Sheep™ 8 Sounds Soother   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
0     Adesso's Rodeo Floor Lamp makes a distinctive addition to your home décor. This accent brightens up any space with an elegant antique bronze finish and textured burlap shade. A 180-degree swivel arm offers illumination wherever you need it. Bring welcoming warmth and light to any space in your home with Adesso's Rodeo Floor Lamp, which puts a stylish urban twist on traditional lamp design Transitional style Textured burlap fabric shade Antique bronze finish Swing arm rotates 180 degrees horizontally Adjust shades vertically and secure with a locking key joint 3-way rotary switch located above the socket Indoor use only Standard outlet Requires one 150 watt medium base bulb (not included) Metal construction Wipe clean Easy to assemble Measures 29\" L x 17\" W x 61\" H Cord measures 60\" L Imported   
1                                                                                                                                                                                                                                                               Add a gentle glow of light during nighttime with the Simple Designs Porcelain Puppy Dog Table Lamp. Made from porcelain, it is adorably shaped like your favorite animal, blending with the decor while adding a soothing touch. Features adorable puppy design Provides a soft light that can be used as a table lamp or night light On/off switch on base Ceramic/fabric/plastic Wipe clean Uses one 25-watt bulb (not included) Plugs into any standard AC outlet Measures 6.49” L 

In [8]:
df['buckets'].explode().value_counts()
len(df)

4787

In [9]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [10]:
df['new_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
n_a=df[df['new_matches'].astype(str)!='[]']

In [11]:
df['new_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_inch, x))
inch=df[df['new_matches'].astype(str)!='[]']

In [12]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
foot=df[df['matches'].astype(str)!='[]']

In [13]:
len(n_a)

102

In [14]:
len(inch)

1307

In [15]:
len(foot)

1154

In [16]:
z=df[(df['matches'].astype(str)!='[]')&(df['new_matches'].astype(str)!='[]')]
z

product_id external_id  \
1350   988737517    69361760   
2639  1111955709    69683279   

                                                                   product_name  \
1350  Kurt Adler 10-Light A Christmas Story™ Colorful Leg Lamp String Light Set   
2639                  HomeRoots Art Glass Table Lamps in White/Green (Set of 2)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
1350                   The Kurt Adler 10-Light A Christmas Story Colorful Leg Lamp String Light Set is a fun and festive addition to your Christmas holiday décor or tree. This light set features the iconic Leg Lamp in several small, colorful designs. Complete your holiday décor with this iconic decorative accent from A Christmas Story Includes: 10 clear incandescent lights 4 spare bulbs Spare fuse For indoor/outdoor use String measures 11' L Cord measures 30\" L Plastic/copper UL listed Imported   
2639  Illuminate your living space with a dazzling look with the HomeRoots Art Glass Table Lamps. Made from glass with a beautiful colorful finish, this pair of stylish lamps are topped with white shades to complete the look. Glass finish and lovely color will enliven any room Set of 2 Rotary socket switch White fabric shade Glass Uses one 150-watt bulb (not included) Plugs into any standard outlet 5' L cord Measures 14\" W x 13\" L x 28\" H Weighs 4.4 lb. 30-day manufacturer's warranty Imported   

      customer_id attributes buckets  bucket_id values            new_matches  \
1350            5       None   Lamps       7916   None  [ Cord measures 30\"]   
2639            5       None   Lamps       7916   None  [ cord Measures 14\"]   

           matches  
1350  [11' L Cord]  
2639   [5' L cord]

In [17]:
inch['new_matches']=inch['new_matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
inch['new_matches']=inch['new_matches'].apply(lambda measure: re.sub(r'\\|\"\,','',str(measure)))
inch['new_matches']=inch['new_matches'].apply(lambda measure: re.sub(r'97\,68','97',str(measure)))
inch['new_matches']=inch['new_matches'].apply(lambda measure: re.sub(r'19\,60','19',str(measure)))
inch['new_matches']=inch['new_matches'].apply(lambda measure: re.sub(r'\:','',str(measure)))
inch.head()

product_id external_id  \
0      4672200    76665915   
17   817472403    61128668   
18   817182203    61128569   
24   817489693    61057654   
29   817471557    60831828   

                                                                                   product_name  \
0                                                     Adesso Rodeo Floor Lamp in Antique Bronze   
17                                                Ren-Wil Valdosta Floor Lamp in Polished Brass   
18                                         Hayton 1-Light Marble Mini Pendant in Polished Brass   
24                                   George Kovacs Exposed Ceiling-Mount Mini Pendant in Chrome   
29  Ambience® by Minka-Lavery® 1-Light Table Lamp with CFL Bulb in Brushed Nickel w/Linen Shade   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  long_desc  \
0   Adesso's Rodeo Floor Lamp makes a distinctive addition to your home décor. This accent brightens up any space with an elegant antique bronze finish and textured burlap shade. A 180-degree swivel arm offers illumination wherever you need it. Bring welcoming warmth and light to any space in your home with Adesso's Rodeo Floor Lamp, which puts a stylish urban twist on traditional lamp design Transitional style Textured burlap fabric shade Antique bronze finish Swing arm rotates 180 degrees horizontally Adjust shades vertically and secure with a locking key joint 3-way rotary switch located above the socket Indoor use only Standard outlet Requires one 150 watt medium base bulb (not included) Metal construction Wipe clean Easy to assemble Measures 29\" L x 17\" W x 61\" H Cord measures 60\" L Imported   
17                                                                                                                                                                                                                                                                                                                     The Valdosta Floor Lamp from Ren-Wil recalls the retro era of 1960s Milan. Soaring from a white marble block base, the metal dome shade hangs on a sweeping metal arch, a glamorous accent over contemporary couches and chairs. The Ren-Wil Valdosta Floor Lamp's dramatic arch, dome shade and marble base bring mid-century glam to modern homes On/off switch Metal and marble construction Uses one 60-watt bulb (not included) Measures 26.8\" L x 10\" W x 61\" H Cord measures 90\" L Imported Model LPF3030   
18                                                                                                                                                                                                                                                              The minimalist design of the Hayton Marhble Mini Pendant makes a bold statement to your décor with a unique exposed bulb. The bell-shaped white marble pendant is finished in a beautiful polished brass for a polished look. The Hayton Marble Mini Pendant features an exposed bulb that offers unique style to your contemporary décor Polished brass finish Ceiling-mount design Must be hardwired Uses 1 medium-base 25-watt incandescent bulb (not included) Marble/Iron Measures 2.4\" diameter x 3.1\" H Cord measures 67\" L Weighs 2.

In [18]:
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r'\\|\"|\,','',str(measure)))
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r'97\,68','97',str(measure)))
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r'19\,60','19',str(measure)))
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r'\:','',str(measure)))
foot

product_id external_id  \
1      817479867    63228465   
2      817192887    63228441   
3      817188038    62520409   
4      817476446    62520393   
5      817187879    62493604   
...          ...         ...   
4796   817567341    65687277   
4808   817293291    64194059   
4811   817288215    63868135   
4813   817545981    63866827   
4817   817281197    63478587   

                                                                                  product_name  \
1                                       Simple Designs Porcelain Puppy Dog Table Lamp in White   
2                                           Simple Designs Porcelain Bunny Table Lamp in White   
3                                                       Décor Therapy Knox Table Lamp in Brown   
4                                                   Décor Therapy Campbell Table Lamp in Brown   
5                                                  Large Brushed Steel Lamp Base with CFL Bulb   
...                                                                                        ...   
4796                         Fangio Lighting Distressed Urn Table Lamps in Metallic (Set of 2)   
4808   Elk Lighting Cynthia 6-Light Ceiling-Mount Pendant in Polished Chrome with Glass Shades   
4811                              Fangio Lighting M.R. Table Lamp in Eggshell with Linen Shade   
4813                                         Fangio Lighting Smooth Scaled Table Lamp in White   
4817  Pacific Coast Lighting 2-Light Hammered Metal Floor Lamp in Brushed Nickel w/Linen Shade   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
1                                                                                        Add a gentle glow of light during nighttime with the Simple Designs Porcelain Puppy Dog Table Lamp. Made from porcelain, it is adorably shaped like your favorite animal, blending with the decor while adding a soothing touch. Features adorable puppy design Provides a soft light that can be used as a table lamp or night light On/off switch on base Ceramic/fabric/plastic Wipe clean Uses one 25-watt bulb (not included) Plugs into any standard AC outlet Measures 6.49” L x 7.28” W x 10.62” H Cord measures 6' L 1-year limited manufacturer's warranty Imported   
2                                                                                                      Add a gentle glow of light during nighttime with the Simple Designs Porcelain Bunny Table Lamp. Made from porcelain, it is adorably shaped like your favorite animal, blending with the decor while adding a soothing touch. Features adorable bunny design Provides a soft light that can be used as a table lamp or night light On/off switch on base Plastic/iron Wipe clean Uses one 25-watt bulb (not included) Plugs into any standard AC outlet Measures 6.49” L x 5.51” W x 10.43” H Cord measures 6' L 1-year limited manufacturer's warranty Imported   
3                                                                                                                                                                                                             Make a statement in your living space with the Knox Table Lamp from Decor Therapy. Full of modern style, this lamp features a poly-resin constructed base of edgy stacked diamonds finished in a rich wood grain, paired with a natural linen drum shade. On/off switch on neck Rich wood resin 

In [19]:
matches_inch=inch
matches_foot=foot

In [20]:
matches_inch[curation_col]=matches_inch['new_matches']
matches_foot[curation_col]=matches_foot['matches']

In [21]:
matches_inch[curation_col]=matches_inch[curation_col].astype(str)
matches_inch[curation_col]=matches_inch[curation_col].apply(lambda x: x+' in')

In [22]:
matches_foot[curation_col]=matches_foot[curation_col].astype(str)
matches_foot[curation_col]=matches_foot[curation_col].apply(lambda x: x+ ' ft')

In [36]:
matches_inch[curation_col]=matches_inch[curation_col].apply(lambda measure: re.sub(r'\"','',str(measure)))
matches_inch[curation_col]=matches_inch[curation_col].apply(lambda measure: re.sub(r'1960','60',str(measure)))
matches_inch[curation_col]=matches_inch[curation_col].apply(lambda measure: re.sub(r'7272','72',str(measure)))
matches_inch[curation_col]=matches_inch[curation_col].apply(lambda measure: re.sub(r'9768','68',str(measure)))
matches_inch

product_id external_id  \
0        4672200    76665915   
17     817472403    61128668   
18     817182203    61128569   
24     817489693    61057654   
29     817471557    60831828   
...          ...         ...   
4812   817545989    63865998   
4814   817545955    63865066   
4816   817543356    63625905   
4819   817277729    62687461   
4830   988575025    40050881   

                                                                                     product_name  \
0                                                       Adesso Rodeo Floor Lamp in Antique Bronze   
17                                                  Ren-Wil Valdosta Floor Lamp in Polished Brass   
18                                           Hayton 1-Light Marble Mini Pendant in Polished Brass   
24                                     George Kovacs Exposed Ceiling-Mount Mini Pendant in Chrome   
29    Ambience® by Minka-Lavery® 1-Light Table Lamp with CFL Bulb in Brushed Nickel w/Linen Shade   
...                                                                                           ...   
4812           Fangio Lighting Stacked Crystal Candlestick Table Lamp in Chrome with Fabric Shade   
4814                             Fangio Lighting Martin Richard Arched Floor Lamp in Satin Nickel   
4816                                           Adesso Piper LED Table Lamp in Black/Antique Brass   
4819                                                 Safavieh Clarabel Table Lamp in White Marble   
4830                                                         Studio 3B™ 18-LED Clip Lamp in Black   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
0     Adesso's Rodeo Floor Lamp makes a distinctive addition to your home décor. This accent brightens up any space with an elegant antique bronze finish and textured burlap shade. A 180-degree swivel arm offers illumination wherever you need it. Bring welcoming warmth and light to any space in your home with Adesso's Rodeo Floor Lamp, which puts a stylish urban twist on traditional lamp design Transitional style Textured burlap fabric shade Antique bronze finish Swing arm rotates 180 degrees horizontally Adjust shades vertically and secure with a locking key joint 3-way rotary switch located above the socket Indoor use only Standard outlet Requires one 150 watt medium base bulb (not included) Metal construction Wipe clean Easy to assemble Measures 29\" L x 17\" W x 61\" H Cord measures 60\" L Imported   
17                                                                                                                                                                                                                                                                                                                       The Valdosta Floor Lamp from Ren-Wil recalls the retro era of 1960s Milan. Soaring from a white marble block base, the metal dome shade hangs on a sweeping metal arch, a glamorous accent over contemporary couches and chairs. The Ren-Wil Valdosta Floor Lamp's dramatic arch, dome shade and marble base bring mid-century glam to modern homes On/off switch Metal and marble construction Uses one 60-watt bulb (not included) Measures 26.8\" L x 10\" W x 61\" H Cord

In [63]:

matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'07','7',str(measure)))
matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'016','16',str(measure)))
matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'09','9',str(measure)))
matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'60 ft','60 in',str(measure)))
matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'68 ft','68 in',str(measure)))
matches_foot[curation_col]=matches_foot[curation_col].apply(lambda measure: re.sub(r'72 ft','72 in',str(measure)))
matches_foot[matches_foot[curation_col].astype(str)=='15 ft']
# matches_foot[curation_col].explode().value_counts()

product_id external_id  \
172    988762293    69604476   
201    988756569    69551461   
479    817285580    63731453   
847    817253814    61159266   
881    988756673    69551453   
1036  1148074528    69689123   
1181   817373680    68265021   
1449   988762289    69604463   
1537   817589582    66995623   
1618   988756559    69551456   
1732   817184329    61520875   
1734     4498696    60655813   
2206     4498697    60655820   
2270   988756566    69551458   
2274   988756675    69551455   
2275   988756674    69551454   
2645   988762295    69604467   
3553   817184324    61520677   
3606   988756563    69551457   
3845   988756568    69551460   
4143   817544350    63731460   
4777   817678092    68296988   

                                                                                  product_name  \
172                                                   Ava Plug-in Pendant Light in Matte White   
201                                               Surya Manning 1-Light Rattan Pendant in Grey   
479                                             Globe Electric James Pendant Fixture in Chrome   
847                                               Adesso® Havana Large Pendant Light in Bronze   
881                                 Surya Matilda 1-Light Downrod Mount Pendant Light in White   
1036                                      Black & Decker™ 3-in1 Handheld Vacuum in Grey/Orange   
1181                        Pacific Coast Lighting® Archie Swing Pharmacy Floor Lamp in Nickel   
1449  Globe Electric Bayfield Plug-In/Hardwire Ceiling Pendant with Glass Shade in Dark Bronze   
1537                                       Globe Electric Downrod Mount Pendant Light in Brass   
1618                                        Surya Mortimer 1-Light Concrete Pendant in Natural   
1732                      Globe Electric Company Skylar 1 Light Plug-In Pendant in Matte Black   
1734                                 Globe Electric 1-Light Angelica Pendant in Chrome/Crystal   
2206                Globe Electric 1-Light Barnyard Plug-In Pendant in Black with 15-Foot Cord   
2270                                           Surya Manning 1-Light Rattan Pendant in Natural   
2274                             Surya Matilda 1-Light Downrod Mount Pendant Light in Charcoal   
2275                                  Surya Matilda 1-Light Downrod Mount Pendant Light in Red   
2645     Globe Electric Sansa Plug-In/Hardwire Ceiling Pendant with Metal Shade in Dark Bronze   
3553                        Globe Electric Edison 1-Light Plug-In Pendants in Black (Set of 3)   
3606                                        Surya Mortimer 1-Light Concrete Pendant in Natural   
3845                                           Surya Manning 1-Light Rattan Pendant in Natural   
4143                                                      Robin Pendant Fixture in Satin Black   
4777                                 BISSELL® Featherweight® Turbo Corded Stick Vacuum in Blue   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           long_desc  \
172             The chic and sleek Ava Plug-in Pendant Light finds its place perfectly in a bedroom or a living room shining brightly on either side of your bed or couch. This clean, modern lamp features a 15' long white cord, so placement is easy. Minimalist design mee

In [23]:
err

NameError: name 'err' is not defined

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [ ]:
# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
       
#         try:
#             if i == 0:
#                 non = not non
#             elif i == -1:
#                 yield int(s) if s.isdigit() else s if case else s.lower()
#                 s = ''
#             else:
#                 x, s = s[:i], s[i:]
#                 yield int(x) if x.isdigit() else x if case else x.lower()
#         except:
#             print(i)

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

In [ ]:
# x['new_matches']= x['matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# x

In [ ]:
# x['new_matches']= x['new_matches'].apply(
#     lambda x: sorted(x, key=natural_key))
# x

In [ ]:
replacement = {
    'a': '',
    'b': '',
    'c': '',
    'd': '',
    'e': '',
    'f': '',
    'g': '',
    'h': '',
    'i': '',
    'j': '',
    'k': '',
    'l': '',
    'm': '',
    'n': '',
    'o': '',
    'p': '',
    'q': '',
    
    'r': '',
    's': '',
    't': '',
    'u': '',
    'v': '',
    'w': '',
    'x': '',
    'y': '',
    'z': '',
    'A': '',
    'k': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
    'K': '',
    'L': '',
    'M': '',
    'N': '',
    'O': '',
    'P': '',
    'Q': '',
    'R': '',
    'S': '',
    'T': '',
    'U': '',
    'V': '',
    'W': '',
    'X': '',
    'Y': '',
    'Z': '',
    '-  ': '',
    ' ': '',
    '  ': '',
    " ": "", 
    "-": "",
    "  ": "",
}

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(5)

In [ ]:
replacement = {
    '\\\"': ' in',
    '\, ' :','
}

In [ ]:
# # Replacement
# x['new_matches']= x['new_matches'].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(5)

In [ ]:
replacement = {
    'in': ' in',

}

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
x.head(5)

In [ ]:
x['matches'][0:500]

In [ ]:
matches=x
matches

In [ ]:
#import regex as re
matches[curation_col]=matches['new_matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\\",'',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"  ",' ',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches

In [ ]:
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"xa0",'',str(measure)))


In [ ]:
matches[0:500]

In [ ]:
len(matches)

In [ ]:
len(matches)

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [66]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/MultipleInch - {attribute}-- match upload_Shawn.csv',index=False)

In [67]:
looks_good('Bed Bath & Beyond', attribute, df, matches_inch)